In [1]:
import csv
import numpy as np
import pandas as pd
import pathlib
import os
from collections import defaultdict

In [2]:

def createAnswerKey(fileName):
    # open AnswerKey csv file and create a dictionary of all the answers, where the format is (Lecture#, question#): Answer
    questionList = []
    answers = {}
    lectureNum = ""
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                questionList = row
                line_count+=1
            else:
                for answerIndex in range(len(row)):
                    if answerIndex > 0:
                        lectureNum = row[0]
                        answer = row[answerIndex]
                        answers[(lectureNum,questionList[answerIndex])] = answer

                line_count+=1

    #print(answers)
    return questionList, answers, lectureNum
                

In [1]:
def cleanAnswer(studAnswer, key):
    try:
        #fixes bug where TRUE/FALSE answers are output at 1.0 or 0.0
        studAnswer = int(float(studAnswer))
        if 'False' in key or 'True' in key:
            if studAnswer == 0:
                return 'False'
            else:
                return 'True'
            
        #turns student answers into integers instead of floats
        else:
            return int(float(studAnswer))
    except:
        return studAnswer

In [4]:
def partAnalysis(fileName, questionList, answers, lectureNum):
   #print('reading file', fileName)
    response = pd.read_csv(fileName)
    analysis = []
    allEmails = response['Email Address'].values
    answeredQuestions = []
    
    for question in response:
        if question in questionList:
            answeredQuestions.append(question)
            
            countCorrect = 0
            countTotal = 0

            studentSubmissions = {}
            index = 0
            
            #iterate through all the student responses for the question
            for studAnswer in response[question]:  
                
                #check if valid/non-null submission
                if not pd.isnull(studAnswer):
                    
                    #assign student to their email
                    student = allEmails[index]
                    
                    #check if this is the student's first valid submission 
                    if student not in studentSubmissions:
                        studentSubmissions[student] = studAnswer

                     
                        studAnswer = cleanAnswer(studAnswer,answers[(lectureNum, question)])
                         
                        #used 'in' instead of '==' to account for whitespace in answers
                        if str(studAnswer) in answers[(lectureNum, question)]:
                            
            
                            countCorrect+=1
     
            
    

                        countTotal +=1

                index+=1

            if countTotal > 0:
                #print(countCorrect,"/",countTotal)

                analysis.append(float(countCorrect/countTotal))
                

    return analysis, answeredQuestions
            
        
   


In [5]:
def listToDict(fullLectureAnalysis, qList):
    #print(qList)
    lectureDict = defaultdict(list)
    for i in range(4):
        for q in range(len(qList)):
            lectureDict[qList[q]].append(fullLectureAnalysis[i][q])
            
    return lectureDict

In [7]:

answerList = []

for path in pathlib.Path("./ANSWERS").iterdir():
    if path.is_file():
        answerList.append(str(path))

for ans in answerList:
    qList, answers, lecNum = createAnswerKey(ans)
    #print(f"------------- Starting {lecNum} -------------")
    lec = lecNum.replace(" ", "").lower()
    #print(lec)
    for path in pathlib.Path("./RESPONSES").iterdir():
        if path.is_dir() and lec in str(path):
            
            
            list_of_files = sorted( filter( lambda x: os.path.isfile(os.path.join(path, x)),
                        os.listdir(path) ) )
    
            fullLectureAnalysis = []
            
            for file in list_of_files:
                analysis, answeredQuestions = partAnalysis(f'{path}/{file}', qList, answers, lecNum)
            
                fullLectureAnalysis.append(analysis)
            
                
            
            lectureDict = listToDict(fullLectureAnalysis, answeredQuestions) 

            df = pd.DataFrame.from_dict(lectureDict, orient = 'index', columns = ['A00', 'B00', 'C00', 'D00'])
            print(df)
            #df.to_csv(f'./RESULTS/{lecNum}.csv')
            print(f"------------- OUTPUT {lecNum} TO CSV -------------")
                


    print()


In [2]:
#### RANDOM TESTS ###
# qList, answers, lecNum = createAnswerKey('./ANSWERS/Lecture Answers - lect3.csv')
# print(answers)

# print("-----------------QUESTIONS-----------------")
# # for q in qList:
# #     print(q)
    
# #path = "./TESTING/testDuplicates.csv"
# path = "./RESPONSES/lecture3/Lecture 3 Participation Form - A00 (Responses).csv"

# analysis, answeredQuestions = partAnalysis(path, qList, answers, lecNum)
# print(answeredQuestions)

# print(analysis)

#print(analysis)
#analysis.to_csv(f'./TESTING/TestB00.csv')
            


NameError: name 'createAnswerKey' is not defined